In [2]:
import glob
import os

Ok - the goal here is to grab all the - first illumina, then nanopore - reads and filter out any human reads from them.  

I think to do this, I think we just align against human (bowtie2 or bwa for illumina and/or minimap2 for nanopore) and keep only unaligned

Alternatively, I could run kraken or centrifuge.  Reading about kraken or centrifuge now to see if they can do this, might be faster.

installed this stuff with conda (bowtie2, samtools, minimap2) - I think I gotta also try bwa mem

First gotta grab and make hg38 index

Note - I found that these guys (https://sites.google.com/site/wiki4metagenomics/tools/short-read/remove-host-sequences) did this too

In [3]:
%cd ~/Data/200725_vre/ref/
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

/mnt/d/Data/200725_vre/ref
--2020-07-25 15:46:05--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  19.2MB/s    in 60s     

2020-07-25 15:47:05 (15.7 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]



Generate bowtie2 Ref

In [5]:
!bowtie2-build --threads 11 hg38.fa.gz hg38

Settings:
  Output files: "hg38.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 44
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  hg38.fa.gz
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:48
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:42
bmax according to bmaxDivN setting: 69302631
Using parameters --bmax 51976974 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 51976974 --dcv 1024
Constructing suffix-array elemen

minimap2 indexing of hg38

In [6]:
!minimap2 -d hg38.mmi hg38.fa.gz

[M::mm_idx_gen::101.387*1.26] collected minimizers
[M::mm_idx_gen::117.104*1.42] sorted minimizers
[M::main::166.536*1.04] loaded/built the index for 455 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 455
[M::mm_idx_stat::167.230*1.04] distinct minimizers: 100202295 (37.96% are singletons); average occurrences: 5.732; average spacing: 5.587
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -d hg38.mmi hg38.fa.gz
[M::main] Real time: 167.792 sec; CPU: 174.172 sec; Peak RSS: 8.808 GB


Ok - now let's test alignment against human for . . . say the first VRE sample

In [26]:
!mkdir ~/Data/200725_vre/ill_proc/
!mkdir ~/Data/200725_vre/nano_proc/

mkdir: cannot create directory ‘/home/timp/Data/200725_vre/ill_proc/’: File exists


In [13]:
illraw='/home/timp/Data/200725_vre/illumina/'
flist=glob.glob(illraw+'*R1*.fastq.gz')
illsamples=[ os.path.basename(filey).split('_')[0] for filey in flist ] 
illsamples

['ENFM',
 'ENFS',
 'KPN',
 'NC',
 'PC1',
 'PC1',
 'PC2',
 'PC2',
 'PC3',
 'PC3',
 'PC4',
 'VRE10',
 'VRE1',
 'VRE2',
 'VRE2',
 'VRE3',
 'VRE4',
 'VRE5',
 'VRE6',
 'VRE7',
 'VRE8',
 'VRE9']

In [35]:
%cd ~/Data/200725_vre/ill_proc/

ref='~/Data/200725_vre/ref/hg38'

for samp in illsamples: 
    r1_samp=glob.glob(illraw+samp+'_*R1*')[0]
    r2_samp=glob.glob(illraw+samp+'_*R2*')[0]
    !bowtie2 -p 8 -x $ref -1 {r1_samp} -2 {r2_samp} | samtools view -f 12 -b | samtools sort -n -o {samp}.unalign.bam
    !samtools fastq -1 {samp}.R1.fq -2 {samp}.R2.fq -n {samp}.unalign.bam

/mnt/d/Data/200725_vre/ill_proc
8614345 reads; of these:
  8614345 (100.00%) were paired; of these:
    8613919 (100.00%) aligned concordantly 0 times
    345 (0.00%) aligned concordantly exactly 1 time
    81 (0.00%) aligned concordantly >1 times
    ----
    8613919 pairs aligned concordantly 0 times; of these:
      1 (0.00%) aligned discordantly 1 time
    ----
    8613918 pairs aligned 0 times concordantly or discordantly; of these:
      17227836 mates make up the pairs; of these:
        17227352 (100.00%) aligned 0 times
        15 (0.00%) aligned exactly 1 time
        469 (0.00%) aligned >1 times
0.01% overall alignment rate
[bam_sort_core] merging from 4 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 17226870 reads
10242293 reads; of these:
  10242293 (100.00%) were paired; of these:
    10240997 (99.99%) aligned concordantly 0 times
    993 (0.01%) aligned concordantly exactly 1 time
    303 (0.00%) aligned concord

Ok - now here's the nanopore alignment

Note - I had to repeat this with fast_q_ instead of fasta, after reextracting the fastq - see my other notebook - I ran that up on duchess (server)

In [6]:
nanoraw='/home/timp/Data/200725_vre/np_fq/'
flist=glob.glob(nanoraw+'*fastq.gz')
nanosamples=[ os.path.basename(filey).split('.')[0] for filey in flist ]
nanosamples

['NC', 'VRE10', 'VRE3', 'VRE5', 'VRE6', 'VRE7']

In [8]:
%cd ~/Data/200725_vre/nano_proc/
ref='~/Data/200725_vre/ref/hg38.mmi'

for samp in nanosamples:
    nanoreads=glob.glob(nanoraw+samp+'*fastq.gz')[0]
    !minimap2 -t 11 -ax map-ont $ref {nanoreads} | samtools view -f 4 | samtools sort -n -o {samp}.unalign.bam
    !samtools fastq {samp}.unalign.bam >{samp}.fq

/mnt/d/Data/200725_vre/nano_proc
[M::main::53.419*0.28] loaded/built the index for 455 target sequence(s)
[M::mm_mapopt_update::60.010*0.33] mid_occ = 728
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 455
[M::mm_idx_stat::60.691*0.33] distinct minimizers: 100202295 (37.96% are singletons); average occurrences: 5.732; average spacing: 5.587
[M::worker_pipeline::109.567*2.69] mapped 139194 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -t 11 -ax map-ont /home/timp/Data/200725_vre/ref/hg38.mmi /home/timp/Data/200725_vre/np_fq/NC.fastq.gz
[M::main] Real time: 110.115 sec; CPU: 294.734 sec; Peak RSS: 7.546 GB
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 72006 reads
[M::main::51.556*0.25] loaded/built the index for 455 target sequence(s)
[M::mm_mapopt_update::53.008*0.27] mid_occ = 728
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 455
[M::mm_idx_stat::53.673*0.28] distinct minimizers: 100202295 (37.96% are singletons); a

In [9]:
!gzip ~/Data/200725_vre/nano_proc/*fq
!gzip ~/Data/200725_vre/ill_proc/*fq

gzip: /home/timp/Data/200725_vre/nano_proc/*fq: No such file or directory


In [12]:
!mkdir ~/Data/200725_vre/vre_sra_up

for samp in illsamples:
    !mv ~/Data/200725_vre/ill_proc/{samp}.R1.fq.gz ~/Data/200725_vre/vre_sra_up/TS_{samp}.ill.R1.fq.gz
    !mv ~/Data/200725_vre/ill_proc/{samp}.R2.fq.gz ~/Data/200725_vre/vre_sra_up/TS_{samp}.ill.R2.fq.gz

for samp in nanosamples:
    !mv ~/Data/200725_vre/nano_proc/{samp}.fq.gz ~/Data/200725_vre/vre_sra_up/TS_{samp}.nano.fq.gz


mkdir: cannot create directory ‘/home/timp/Data/200725_vre/vre_sra_up’: File exists
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/PC1.R1.fq.gz': No such file or directory
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/PC1.R2.fq.gz': No such file or directory
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/PC2.R1.fq.gz': No such file or directory
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/PC2.R2.fq.gz': No such file or directory
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/PC3.R1.fq.gz': No such file or directory
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/PC3.R2.fq.gz': No such file or directory
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/VRE2.R1.fq.gz': No such file or directory
mv: cannot stat '/home/timp/Data/200725_vre/ill_proc/VRE2.R2.fq.gz': No such file or directory
